In [101]:
# Import libraries
from pathlib import Path
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime as dt
import yfinance as yf

%matplotlib inline

In [102]:
# Read csv into pandas df
sp_csv = pd.read_csv(Path("sp500.csv"))
sp_csv.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,ABIOMED Inc,Health Care


In [103]:
# Create dataframe with S&P 500 tickers based on information in CSV file
sp_csv = pd.read_csv(Path("sp500.csv"))

In [5]:
# <Insert Comment>
tickers = [x for x in sp_csv.loc[:, "Symbol"]]

# Appending SPY data
tickers.append("SPY")

# Downloading historical price data for S&P 500 securities using Yfinance library
data = yf.download(tickers=tickers, start="2012-01-01", end="2017-01-01", group_by="ticker")

[*********************100%***********************]  506 of 506 completed

10 Failed downloads:
- FOXA: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- BF.B: No data found for this date range, symbol may be delisted
- IR: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- CARR: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- FOX: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- CTVA: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- VIAC: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- DOW: Data doesn't exist for startDate = 1325394000, endDate = 1483246800
- BRK.B: No data found, symbol may be delisted
- OTIS: Data doesn't exist for startDate = 1325394000, endDate = 1483246800


In [104]:
data.head()

,CMG,TIF,LKQ,EW,CDNS,CI,MYL,MAS,CHD,GD,...,TROW,MSCI,TWTR,LEN,EMN,SPG,MCO,TJX,JNJ,GM
,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,...,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,341.269989,66.480003,15.390,12.000000,10.34,43.230000,22.150000,9.648506,22.700001,68.000000,...,57.590000,32.810001,NaN,19.557522,40.200001,121.712135,35.160000,15.927500,65.879997,21.049999
2012-01-04,348.750000,66.790001,15.480,11.806666,10.06,43.209999,21.740000,9.833040,22.410000,68.120003,...,57.480000,32.349998,NaN,19.872173,40.090000,119.604889,35.070000,16.107500,65.480003,21.150000
2012-01-05,350.480011,66.680000,15.800,11.991667,10.37,43.209999,21.490000,10.140597,22.575001,67.400002,...,58.290001,32.720001,NaN,20.422812,40.099998,120.235184,35.689999,16.577499,65.400002,22.170000
2012-01-06,348.950012,65.750000,15.615,12.146667,10.19,43.610001,21.469999,10.017574,22.930000,67.620003,...,58.709999,33.099998,NaN,20.058996,41.320000,119.303856,35.830002,16.615000,64.830002,22.920000
2012-01-09,339.739990,66.940002,15.655,12.105000,10.05,43.779999,21.709999,10.017574,22.969999,68.370003,...,58.200001,32.970001,NaN,20.334316,41.450001,119.209785,35.830002,16.424999,64.930000,22.840000


In [105]:
def clean_yahoo_df(data):
    data.drop(["Open", "High", "Low", "Adj Close", "Volume"], axis=1, level=1, inplace=True)
    df_clean = data.droplevel(axis=1, level=1)
    return df_clean
    
sp_df_clean = clean_yahoo_df(data)
sp_df_clean.head()

KeyError: "labels ['Open' 'High' 'Low' 'Adj Close' 'Volume'] not found in level"

In [8]:
sp_df_clean['SPY']

Date
2012-01-03    127.500000
2012-01-04    127.699997
2012-01-05    128.039993
2012-01-06    127.709999
2012-01-09    128.020004
                 ...    
2016-12-23    225.710007
2016-12-27    226.270004
2016-12-28    224.399994
2016-12-29    224.350006
2016-12-30    223.529999
Name: SPY, Length: 1260, dtype: float64

In [9]:
sp_df_clean.index

DatetimeIndex(['2012-01-03', '2012-01-04', '2012-01-05', '2012-01-06',
               '2012-01-09', '2012-01-10', '2012-01-11', '2012-01-12',
               '2012-01-13', '2012-01-17',
               ...
               '2016-12-16', '2016-12-19', '2016-12-20', '2016-12-21',
               '2016-12-22', '2016-12-23', '2016-12-27', '2016-12-28',
               '2016-12-29', '2016-12-30'],
              dtype='datetime64[ns]', name='Date', length=1260, freq=None)

In [10]:
sp_df_clean.to_csv("sp500_raw_data.csv")

In [106]:
sp_path = Path("sp500_raw_data.csv")
sp_combined_df = pd.read_csv(sp_path, index_col='Date',infer_datetime_format=True,parse_dates=True)
sp_combined_df.head(5)

,CMG,TIF,LKQ,EW,CDNS,CI,MYL,MAS,CHD,GD,...,TROW,MSCI,TWTR,LEN,EMN,SPG,MCO,TJX,JNJ,GM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,341.269989,66.480003,15.390,12.000000,10.34,43.230000,22.150000,9.648506,22.700001,68.000000,...,57.590000,32.810001,NaN,19.557522,40.200001,121.712135,35.160000,15.927500,65.879997,21.049999
2012-01-04,348.750000,66.790001,15.480,11.806666,10.06,43.209999,21.740000,9.833040,22.410000,68.120003,...,57.480000,32.349998,NaN,19.872173,40.090000,119.604889,35.070000,16.107500,65.480003,21.150000
2012-01-05,350.480011,66.680000,15.800,11.991667,10.37,43.209999,21.490000,10.140597,22.575001,67.400002,...,58.290001,32.720001,NaN,20.422812,40.099998,120.235184,35.689999,16.577499,65.400002,22.170000
2012-01-06,348.950012,65.750000,15.615,12.146667,10.19,43.610001,21.469999,10.017574,22.930000,67.620003,...,58.709999,33.099998,NaN,20.058996,41.320000,119.303856,35.830002,16.615000,64.830002,22.920000
2012-01-09,339.739990,66.940002,15.655,12.105000,10.05,43.779999,21.709999,10.017574,22.969999,68.370003,...,58.200001,32.970001,NaN,20.334316,41.450001,119.209785,35.830002,16.424999,64.930000,22.840000


In [107]:
# Check to confirm whether there are any duplicates in the index variable
sp_combined_df.duplicated()

Date
2012-01-03    False
2012-01-04    False
2012-01-05    False
2012-01-06    False
2012-01-09    False
              ...  
2016-12-23    False
2016-12-27    False
2016-12-28    False
2016-12-29    False
2016-12-30    False
Length: 1260, dtype: bool

In [108]:
sp_combined_df.to_csv("sp500_clean.csv")

In [109]:
sp_csv_clean = pd.read_csv(Path("sp500_clean.csv"),index_col='Date',infer_datetime_format=True,parse_dates=True)
sp_csv_clean.head()

,CMG,TIF,LKQ,EW,CDNS,CI,MYL,MAS,CHD,GD,...,TROW,MSCI,TWTR,LEN,EMN,SPG,MCO,TJX,JNJ,GM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,341.269989,66.480003,15.390,12.000000,10.34,43.230000,22.150000,9.648506,22.700001,68.000000,...,57.590000,32.810001,NaN,19.557522,40.200001,121.712135,35.160000,15.927500,65.879997,21.049999
2012-01-04,348.750000,66.790001,15.480,11.806666,10.06,43.209999,21.740000,9.833040,22.410000,68.120003,...,57.480000,32.349998,NaN,19.872173,40.090000,119.604889,35.070000,16.107500,65.480003,21.150000
2012-01-05,350.480011,66.680000,15.800,11.991667,10.37,43.209999,21.490000,10.140597,22.575001,67.400002,...,58.290001,32.720001,NaN,20.422812,40.099998,120.235184,35.689999,16.577499,65.400002,22.170000
2012-01-06,348.950012,65.750000,15.615,12.146667,10.19,43.610001,21.469999,10.017574,22.930000,67.620003,...,58.709999,33.099998,NaN,20.058996,41.320000,119.303856,35.830002,16.615000,64.830002,22.920000
2012-01-09,339.739990,66.940002,15.655,12.105000,10.05,43.779999,21.709999,10.017574,22.969999,68.370003,...,58.200001,32.970001,NaN,20.334316,41.450001,119.209785,35.830002,16.424999,64.930000,22.840000


In [110]:
# New Code Starts Here
sp_combined_df.shape

(1260, 506)

In [111]:
sp_combined_df.isnull().sum()

CMG     2
TIF     2
LKQ     2
EW      2
CDNS    2
       ..
SPG     2
MCO     2
TJX     2
JNJ     2
GM      2
Length: 506, dtype: int64

In [112]:
# Exploring data frame in preparation for cleaning of data
sp_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1260 entries, 2012-01-03 to 2016-12-30
Columns: 506 entries, CMG to GM
dtypes: float64(506)
memory usage: 4.9 MB


In [113]:
# Confirming data types in dataframe are all float values
sp_combined_df.dtypes

CMG     float64
TIF     float64
LKQ     float64
EW      float64
CDNS    float64
         ...   
SPG     float64
MCO     float64
TJX     float64
JNJ     float64
GM      float64
Length: 506, dtype: object

In [114]:
# Confirming maximum amount of data, i.e. number of trading days
# that will be included in the period is 1258
sp_combined_df.count()

CMG     1258
TIF     1258
LKQ     1258
EW      1258
CDNS    1258
        ... 
SPG     1258
MCO     1258
TJX     1258
JNJ     1258
GM      1258
Length: 506, dtype: int64

In [115]:
# Checking for null values in data set (anticipate there will be some)
# with stocks that were not present in the S&P 500 for the full time

sp_combined_df.isnull()

,CMG,TIF,LKQ,EW,CDNS,CI,MYL,MAS,CHD,GD,...,TROW,MSCI,TWTR,LEN,EMN,SPG,MCO,TJX,JNJ,GM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2012-01-04,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2012-01-05,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2012-01-06,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2012-01-09,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2016-12-27,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2016-12-28,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [116]:
sp_combined_df.isnull().mean() *100

CMG     0.15873
TIF     0.15873
LKQ     0.15873
EW      0.15873
CDNS    0.15873
         ...   
SPG     0.15873
MCO     0.15873
TJX     0.15873
JNJ     0.15873
GM      0.15873
Length: 506, dtype: float64

In [117]:
# Cleaning data - given numeric in nature, all NaN values
df_all_no_null=sp_combined_df.dropna(thresh=2).copy()

In [118]:
# Confirm cleanse of data worked by re-running check for null values
df_all_no_null.isnull().sum()

CMG     0
TIF     0
LKQ     0
EW      0
CDNS    0
       ..
SPG     0
MCO     0
TJX     0
JNJ     0
GM      0
Length: 506, dtype: int64

In [120]:
df_all_no_null.head()

,CMG,TIF,LKQ,EW,CDNS,CI,MYL,MAS,CHD,GD,...,TROW,MSCI,TWTR,LEN,EMN,SPG,MCO,TJX,JNJ,GM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,341.269989,66.480003,15.390,12.000000,10.34,43.230000,22.150000,9.648506,22.700001,68.000000,...,57.590000,32.810001,NaN,19.557522,40.200001,121.712135,35.160000,15.927500,65.879997,21.049999
2012-01-04,348.750000,66.790001,15.480,11.806666,10.06,43.209999,21.740000,9.833040,22.410000,68.120003,...,57.480000,32.349998,NaN,19.872173,40.090000,119.604889,35.070000,16.107500,65.480003,21.150000
2012-01-05,350.480011,66.680000,15.800,11.991667,10.37,43.209999,21.490000,10.140597,22.575001,67.400002,...,58.290001,32.720001,NaN,20.422812,40.099998,120.235184,35.689999,16.577499,65.400002,22.170000
2012-01-06,348.950012,65.750000,15.615,12.146667,10.19,43.610001,21.469999,10.017574,22.930000,67.620003,...,58.709999,33.099998,NaN,20.058996,41.320000,119.303856,35.830002,16.615000,64.830002,22.920000
2012-01-09,339.739990,66.940002,15.655,12.105000,10.05,43.779999,21.709999,10.017574,22.969999,68.370003,...,58.200001,32.970001,NaN,20.334316,41.450001,119.209785,35.830002,16.424999,64.930000,22.840000


In [121]:
df_all_clean = df_all_no_null.dropna(axis=1)
df_all_clean.head()

,CMG,TIF,LKQ,EW,CDNS,CI,MYL,MAS,CHD,GD,...,SYY,TROW,MSCI,LEN,EMN,SPG,MCO,TJX,JNJ,GM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,341.269989,66.480003,15.390,12.000000,10.34,43.230000,22.150000,9.648506,22.700001,68.000000,...,29.450001,57.590000,32.810001,19.557522,40.200001,121.712135,35.160000,15.927500,65.879997,21.049999
2012-01-04,348.750000,66.790001,15.480,11.806666,10.06,43.209999,21.740000,9.833040,22.410000,68.120003,...,29.000000,57.480000,32.349998,19.872173,40.090000,119.604889,35.070000,16.107500,65.480003,21.150000
2012-01-05,350.480011,66.680000,15.800,11.991667,10.37,43.209999,21.490000,10.140597,22.575001,67.400002,...,28.910000,58.290001,32.720001,20.422812,40.099998,120.235184,35.689999,16.577499,65.400002,22.170000
2012-01-06,348.950012,65.750000,15.615,12.146667,10.19,43.610001,21.469999,10.017574,22.930000,67.620003,...,28.920000,58.709999,33.099998,20.058996,41.320000,119.303856,35.830002,16.615000,64.830002,22.920000
2012-01-09,339.739990,66.940002,15.655,12.105000,10.05,43.779999,21.709999,10.017574,22.969999,68.370003,...,28.910000,58.200001,32.970001,20.334316,41.450001,119.209785,35.830002,16.424999,64.930000,22.840000


In [122]:
df_all_clean.to_csv("final_clean_df.csv")

In [124]:
# Creating portfolio that is just the S&P 500 returns
portfolio_sp500 = df_all_clean.loc[:,'SPY']
portfolio_sp500.head(5)

Date
2012-01-03    127.500000
2012-01-04    127.699997
2012-01-05    128.039993
2012-01-06    127.709999
2012-01-09    128.020004
Name: SPY, dtype: float64

In [ ]:
# Creating portfolio that is made up of the 50 highest volaility stocks
portfolio_high_vol = df_all_no_null.nlargest(50, 'Volatility')
portfolio_high_vol.head(5)

In [ ]:
# Creating portfolio that is made up of the 50 smallest volaility stocks
portfolio_low_vol = df_all_no_null.nsmallest(50, 'Volatility')
portfolio_low_vol.head(5)

In [32]:
# Create list of random numbers 1-485 and loop through list in order to create dataframe of random stocks
import random

random_num_list = random.sample(range(400), 20)
print(random_num_list)

[177, 37, 327, 343, 148, 384, 344, 54, 379, 273, 219, 316, 111, 5, 7, 66, 257, 225, 113, 387]


In [ ]:
portfolio_random = []

for i in random_num_list:
    porfolio_random.concatenate[]